In [ ]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook, load_workbook
import pandas as pd

# Configurações do Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=chrome_options)

# URL do site da FGV
page_url = "https://extra-ibre.fgv.br/autenticacao_produtos_licenciados/?ReturnUrl=%2fautenticacao_produtos_licenciados%2flista-produtos.aspx"
driver.get(page_url)
print("Site abriu com sucesso")

# Esperar até o botão 'fechar dúvidas' ser clicável
wait = WebDriverWait(driver, 10)
button_serieinstitucional = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/form/div[3]/div/div/div[2]/div[3]/a[2]')))
button_serieinstitucional.click()
print("Botão 'series institucionais' clicado.")

# Interação com o formulário
datainicial = wait.until(EC.element_to_be_clickable((By.ID, 'txtBuscarSeries')))
datainicial.clear()
datainicial.send_keys("Índice Geral de Preços - IGP-M")
print("Digitado IGPM")

button_ok = wait.until(EC.element_to_be_clickable((By.ID, 'butBuscarSeries')))
button_ok.click()
print("Botão 'OK' clicado.")

button_selecionartodos = wait.until(EC.element_to_be_clickable((By.ID, 'btnSelecionarTodas')))
button_selecionartodos.click()  
print("Botão 'selecionar todos' clicado.")
time.sleep(2)

button_visualizar = wait.until(EC.element_to_be_clickable((By.ID, 'butBuscarSeriesOK')))
button_visualizar.click()
print("Botão 'OK' clicado.")

button_serieihistorica = wait.until(EC.element_to_be_clickable((By.ID, 'cphConsulta_rbtSerieHistorica')))
button_serieihistorica.click()
print("Botão 'serie histórica' clicado.")

button_visualizar = wait.until(EC.element_to_be_clickable((By.ID, 'cphConsulta_butVisualizarResultado')))
button_visualizar.click()
print("Botão 'OK' clicado.")

# Aguardar até que o iframe esteja disponível
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'cphConsulta_ifrVisualizaConsulta')))

link_csv = wait.until(EC.element_to_be_clickable((By.ID, 'lbtSalvarCSV')))
link_csv.click()
print("Arquivo baixado")

# Aguardar o download concluir (pode usar um método mais robusto)
time.sleep(30)

# Identificar o arquivo mais recente na pasta de downloads
downloads_path = r"C:\Users\DXX7\Downloads"
arquivos = os.listdir(downloads_path)
arquivos_csv = [arquivo for arquivo in arquivos if arquivo.endswith(".csv")]
arquivo_mais_recente = max(arquivos_csv, key=lambda x: os.path.getctime(os.path.join(downloads_path, x)))

# Caminho completo para o arquivo mais recente
caminho_arquivo = os.path.join(downloads_path, arquivo_mais_recente)

# Caminho de destino do arquivo Excel
caminho_destino = r"xxxx"

# Abrir o arquivo e salvar no CSV
try:
    file = pd.read_table(caminho_arquivo, sep=";", encoding='ISO-8859-1')  # Tentar com ISO-8859-1
    file.to_csv(caminho_destino, index=False, sep=';', encoding='utf-8-sig')
    print("Arquivo convertido e salvo em:", caminho_destino)
except Exception as e:
    print("Erro ao processar o arquivo:", e)



print("IGPM TRANSFERIDO para", caminho_destino)

# Fechar o navegador
driver.quit()
print("Navegador fechado.")



import os
import pandas as pd
from datetime import datetime

# Caminho do arquivo CSV de entrada e saída
input_csv_path = r'xxxx'

# Verificar se o arquivo CSV de entrada existe
if not os.path.exists(input_csv_path):
    print(f'Arquivo CSV de entrada não encontrado em: {input_csv_path}')
    exit()

# Lista de encodings a serem tentados
encodings = ['utf-8', 'latin-1', 'iso-8859-1']

# Ler o arquivo CSV
df = None
for encoding in encodings:
    try:
        df = pd.read_csv(input_csv_path, sep=';', encoding=encoding)
        print(f'Arquivo lido com sucesso. Cabeçalho:\n{df.head()}')
        break  # Se conseguiu ler, interrompe o loop
    except Exception as e:
        print(f'Erro ao ler o arquivo CSV com encoding {encoding}: {e}')

if df is None:
    print(f'Não foi possível ler o arquivo CSV com os encodings fornecidos.')
    exit()

# Garantir que a coluna 'Data' esteja no formato datetime
# Ajustar para o formato de data "mês/ano"
df['Data'] = pd.to_datetime(df['Data'].str.strip(), format='%m/%Y', errors='coerce')

# Verificar se a conversão da data foi bem-sucedida
if df['Data'].isnull().all():
    print("Não foi possível encontrar uma data válida no DataFrame.")
    exit()

print("Valores únicos da coluna 'Data' após conversão:\n", df['Data'].unique())

# Criar novo DataFrame com os dados necessários
ultimo_dado = df['Data'].max()  # Encontrar a data máxima

summary_df = pd.DataFrame({
    'codigo': ['IGPM'],
    'nome_completo': ['Índice Geral de Preços do Mercado - IGPM'],
    'Data_do_ultimo_dado': [ultimo_dado.strftime('%d/%m/%Y') if pd.notnull(ultimo_dado) else 'N/A'],
    'Data_de_atualizacao_da_base': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
})

# Verificar o DataFrame resumido antes de salvar
print("Resumo do DataFrame:\n", summary_df)

# Definir caminho para o novo arquivo CSV
output_summary_csv_path = os.path.join(os.path.dirname(input_csv_path), 'IGPM_resumo.csv')

# Salvar o resumo em um novo arquivo CSV
try:
    summary_df.to_csv(output_summary_csv_path, index=False, sep=';', encoding='utf-8-sig')
    print(f'Dados resumidos salvos em: {output_summary_csv_path}')
except Exception as e:
    print(f'Erro ao salvar dados resumidos no arquivo CSV: {e}')



Site abriu com sucesso
Botão 'series institucionais' clicado.
Digitado IGPM
Botão 'OK' clicado.
Botão 'selecionar todos' clicado.
Botão 'OK' clicado.
Botão 'serie histórica' clicado.
Botão 'OK' clicado.
Arquivo baixado
Arquivo convertido e salvo em: C:\Users\FNEQ\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\FGV\IGP-M_saida.csv
IGPM TRANSFERIDO para C:\Users\FNEQ\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\FGV\IGP-M_saida.csv
Navegador fechado.
Arquivo lido com sucesso. Cabeçalho:
      Data Índice Geral de Preços - IGP-M(200045)
0  06/1989                          3,086190E-005
1  07/1989                          4,194290E-005
2  08/1989                          5,742660E-005
3  09/1989                          8,034990E-005
4  10/1989                          1,130011E-004
Valores únicos da coluna 'Data' após conversão:
 <DatetimeArray>
['1989-06-01 00:00:00', '1989-07-01 00:00:00', '1989-08-01 00:00:00',
 '1989-09-01 00:00:00', '1989-10-01 00:00:00', '1989-